In [1]:
# mac475의 ipython 표준 style을 적용함
from IPython.core.display import HTML
styles = open("../styles/custom.css", "r").read()
HTML( styles )

    1. tube의 정리
    2. tube의 주요 feature 검사
        2.1 tube 누락 feature 확인
        2.2 material id 확인
        2.3 numeric feature overview

        2.4 bend_radius의 보완
                2.4.1 bend radius를 위한 train/ test dataset의 구성
                2.4.2 Label Encoding 작업
                2.4.3 training data에 대한 7개 feature 대상 encoding 수행
                2.4.4 train dataset의 encoded 상태 확인
                2.4.5 Modeling 위한 X, y 생성
                2.4.6 Modeling 수행
                2.4.7 test data에 대한 7개 feature 대상 encoding 수행
                2.4.8 bend_radius prediction 수행
                2.4.9 Prediction 결과를 tube에 value assign 수행
                2.4.10 prediction 반영결과를 output

        2.5 end_x의 보완
                2.5.1 end_x를 위한 train/ test dataset의 구성
                2.5.2 Label Encoding 작업
                2.5.3 training data에 대한 7개 feature 대상 encoding 수행
                2.5.4 train dataset의 encoded 상태 확인
                2.5.5 Modeling 위한 X, y 생성
                2.5.6 Modeling 수행
                2.5.7 test data에 대한 7개 feature 대상 encoding 수행
                2.5.8 end_x prediction 수행
                2.5.9 Prediction 결과를 tube에 value assign 수행
                2.5.10 prediction 반영결과를 output

        2.6 material_id의 보완
                2.6.1 material_id를 위한 train/ test dataset의 구성
                2.6.2 Label Encoding 작업
                2.6.3 training data에 대한 7개 feature 대상 encoding 수행
                2.6.4 train dataset의 encoded 상태 확인
                2.6.5 Modeling 위한 X, y 생성
                2.6.6 Modeling 수행
                2.6.7 test data에 대한 7개 feature 대상 encoding 수행
                2.6.8 material_id prediction 수행
                2.6.9 Prediction 결과를 tube에 value assign 수행
                2.6.10 prediction 반영결과를 output

#1. tube의 정리
- tube dataset의 정리작업 필요성

    → train dataset만 활용시 modeling시 활용가능 feature 제약, tube_assembly_id를 join하여 tube dataset내의 tube 속성정보 modeling에 활용
    
    → material_id, diameter, wall, length, num_bends, bend_radius, end관련 6개 속성, num_boss, num_bracket, other
- 단, tube dataset을 보완필요 있는지 사전확인후, 적절한 작업의 병행필요

<img src="images/01.train-tube.png" style="display:inline; width: 30%" />

In [2]:
import pandas as pd
import numpy as np

# 당초 제공되었던 tube dataset중 length == 0에 대한 admin측의 보완에 의해 보정된 dataset임
df = pd.read_csv( './dataset/01.original.dataset/tube.length.verified.csv' )
# df = pd.read_csv( './dataset/01.original.dataset/tube.length.training.verified.csv' )

#2. tube의 주요 feature 검사

##2.1 tube 누락 feature 확인

In [3]:
df.count()    # tube의 전체적인 data 충실도를 확인

tube_assembly_id    21198
material_id         20919
diameter            21198
wall                21198
length              21198
num_bends           21198
bend_radius         21198
end_a_1x            21198
end_a_2x            21198
end_x_1x            21198
end_x_2x            21198
end_a               21198
end_x               21198
num_boss            21198
num_bracket         21198
other               21198
dtype: int64

<font color = 'red'>* 확인결과 : 모든 data는 충실하나, material_id에 279개 누락이 있다 → 보완필요함</font>

.

##2.2 material id 확인

In [4]:
# 보완이 필요한 material의 data 구성을 아래와 같이 살펴본다
df_material = pd.DataFrame( data = df[ 'material_id' ].value_counts(), columns = [ 'cum_sum' ] )
df_material[ 'pct' ] = df_material[ 'cum_sum' ] / df_material[ 'cum_sum' ].sum() * 100
df_material[ 'cum_pct' ] = df_material[ 'cum_sum' ].cumsum() / df_material[ 'cum_sum' ].sum() * 100
df_material.head( 3 )

,cum_sum,pct,cum_pct
SP-0029,11488,54.916583,54.916583
SP-0035,3676,17.572542,72.489125
SP-0028,1644,7.858884,80.348009


In [5]:
del( df_material )

<font color = 'red'>* 확인결과 : 50% 이상이 SP-0029이고, 상위 7개가 95% 이상을 점유하고 있다</font>

.

##2.3 numeric feature overview
###대상 : diameter, wall, length, num_bends, bend_radius, num_boss, num_bracket, other

In [6]:
df.describe()

,diameter,wall,length,num_bends,bend_radius,num_boss,num_bracket,other
count,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000
mean,23.687764,1.578402,100.337469,3.748137,49.350767,0.044485,0.006746,0.022644
std,25.396277,0.745749,70.520780,2.265405,196.468480,0.260737,0.100015,0.199676
min,3.180000,0.710000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.520000,0.890000,48.000000,2.000000,19.050000,0.000000,0.000000,0.000000
50%,15.880000,1.650000,86.000000,3.000000,38.100000,0.000000,0.000000,0.000000
75%,25.400000,1.650000,135.000000,5.000000,50.800000,0.000000,0.000000,0.000000
max,203.200000,7.900000,1333.000000,17.000000,9999.000000,5.000000,5.000000,8.000000


* 확인결과
    
    diameter : 3.18 ~ 203.20
    
    wall : 0.71 ~ 7.90
    
    length : 1 ~ 1,333
    
    num_bends : 0 ~ 17 : 굽힘의 개수로 추정, 0개일 수 있다고 판단
    
    bend_radius : 0 ~ 9999 : 9999는 뭘까, 확인필요 → Radius 0 means no bend.  Radius 9999 means unknown
    
    num_boss : 0 ~ 5, ?의 개수
    
    num_bracket : 0 ~ 5, 브라켓의 개수
    
    other : 0 ~ 8, 기타의 개수
    
    Kaggle Forum 참고['https://www.kaggle.com/c/caterpillar-tube-pricing/forums/t/15048/zero-length-tubes/83438']

.

##이상하다 판단된 9999 unknown case의 경우, 만약 제외한게 되면 test_set과 merge시 같이 누락되어 버리므로 버릴 수 없고, 어떻게든 살려서 활용해야 함

##2.4 bend_radius의 보완
###bend_radius == 9999인경우, unknown이므로 이를 확인해야 함
####8개 data에 대해 발생했음 : 간단한 ML통해 이를 보완하기로 함

In [7]:
len( df[ df[ 'bend_radius' ] > 9000 ] )    # bend_radius가 unknown인 개수

8

In [8]:
df[ df[ 'bend_radius' ] > 9000 ]    # bend_radius를 prediction해야하는 test dataset이다

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
15132,TA-15133,SP-0029,31.75,1.65,49,2,9999,N,N,N,N,EF-003,EF-003,0,0,0
15174,TA-15175,SP-0029,31.75,1.65,36,1,9999,N,N,N,N,EF-003,EF-003,0,0,0
15175,TA-15176,SP-0029,31.75,1.65,76,1,9999,N,N,N,N,EF-003,EF-003,0,0,0
17688,TA-17689,SP-0046,66.70,7.90,88,2,9999,N,Y,N,N,NONE,NONE,0,0,0
17689,TA-17690,SP-0046,66.70,7.90,71,2,9999,N,Y,N,N,NONE,NONE,0,0,0
18002,TA-18003,SP-0046,66.70,7.90,63,2,9999,N,Y,N,Y,NONE,NONE,0,0,0
18003,TA-18004,SP-0046,66.70,7.90,81,2,9999,N,Y,N,N,NONE,NONE,0,0,0
19320,TA-19321,SP-0019,6.35,0.71,32,3,9999,N,N,N,N,EF-008,EF-008,0,0,0


* bend_radius == 9999로 unknown인 것들은 다행히 material_id가 존재한다

###2.4.1 bend radius를 위한 train/ test dataset의 구성

In [9]:
# training dataset 구성시, material이  NA인 것은 제외하고 진행한다
df_bend_radius_train = df[ ( df[ 'bend_radius' ] != 9999 ) & ( df[ 'material_id' ].notnull() ) ]    # 정상 dataset을 train
df_bend_raius_test = df[ df[ 'bend_radius' ] == 9999 ]    # 9999 dataset을 test

###2.4.2 Label Encoding 작업
  * categorical data를 모두 포함해야 하므로, df를 기반으로 label encoder를 fitting한다
  * label encoding의 대상은 materian_id, end_a_1x, end_a_2x, end_x_1x, end_x_2x, end_a, end_x 7개 feature이다
  * 단, material_id의 경우, df의 상태로는 NA가 포함되어 있으므로, 이를 제거하고 encoding 수행

In [10]:
from sklearn import preprocessing

# Label Encoding 필요한 feature들중, material_id, end_a, end_x는 불완전하므로, null, none 등 check 활용 수행
le_material_id = preprocessing.LabelEncoder()
le_material_id.fit( df[ 'material_id' ][ df[ 'material_id' ].notnull() ] )    # notnull()로 filtering 가능

le_end_a_1x = preprocessing.LabelEncoder()
le_end_a_1x.fit( df[ 'end_a_1x' ] )    # filtering 필요없음

le_end_a_2x = preprocessing.LabelEncoder()
le_end_a_2x.fit( df[ 'end_a_2x' ] )    # filtering 필요없음

le_end_x_1x = preprocessing.LabelEncoder()
le_end_x_1x.fit( df[ 'end_x_1x' ] )    # filtering 필요없음

le_end_x_2x = preprocessing.LabelEncoder()
le_end_x_2x.fit( df[ 'end_x_2x' ] )    # filtering 필요없음

le_end_a = preprocessing.LabelEncoder()
le_end_a.fit( df[ 'end_a' ]  )    # notnull() filtering 불가, 'NONE' filtering 필요

le_end_x = preprocessing.LabelEncoder()
le_end_x.fit( df[ 'end_x' ] )    # notnull() filtering 불가, 'NONE' filtering 필요

LabelEncoder()

###2.4.3 training data에 대한 7개 feature 대상 encoding 수행

In [11]:
df_bend_radius_train[ 'material_id' ] = le_material_id.transform( df_bend_radius_train[ 'material_id' ] )

df_bend_radius_train[ 'end_a_1x' ] = le_end_a_1x.transform( df_bend_radius_train[ 'end_a_1x' ] )
df_bend_radius_train[ 'end_a_2x' ] = le_end_a_2x.transform( df_bend_radius_train[ 'end_a_2x' ] )

df_bend_radius_train[ 'end_x_1x' ] = le_end_x_1x.transform( df_bend_radius_train[ 'end_x_1x' ] )
df_bend_radius_train[ 'end_x_2x' ] = le_end_x_2x.transform( df_bend_radius_train[ 'end_x_2x' ] )

df_bend_radius_train[ 'end_a' ] = le_end_a.transform( df_bend_radius_train[ 'end_a' ] )
df_bend_radius_train[ 'end_x' ] = le_end_x.transform( df_bend_radius_train[ 'end_x' ] )

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydat

###2.4.4 train dataset의 encoded 상태 확인

In [12]:
df_bend_radius_train.head( 3 )
# 이것 활용하여, bend_radius == 9999를 보정하기 위한, regression model을 만들어야 함

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,TA-00001,9,12.70,1.65,164,5,38.10,0,0,0,0,2,3,0,0,0
1,TA-00002,1,6.35,0.71,137,8,19.05,0,0,0,0,7,8,0,0,0
2,TA-00003,1,6.35,0.71,127,7,19.05,0,0,0,0,7,8,0,0,0


###2.4.5 Modeling 위한 X, y 생성

In [13]:
removal_feature_list = [
#                          'material_id',
#                          'diameter',
#                          'wall',
#                          'length',
#                          'num_bends',
    
                         'end_a_1x',
                         'end_a_2x',
                         'end_x_1x',
                         'end_x_2x',
                         'end_a',
                         'end_x',
    
#                          'num_boss',
#                          'num_bracket',
#                          'other',
                       ]

In [14]:
df_bend_radius_train.drop( removal_feature_list, axis = 1, inplace = True )    # features engineered

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [15]:
X = df_bend_radius_train.drop( [ 'tube_assembly_id', 'bend_radius' ], axis = 1 )    # X를 생성한다
y = df_bend_radius_train[ 'bend_radius' ]    # y를 생성한다

###2.4.6 Modeling 수행

In [16]:
cv_cnt = 10    # cv : cross validation 횟수
n_jobs_cnt = 6

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from sklearn.ensemble import RandomForestRegressor

from sklearn import grid_search

In [18]:
model_list = [
#                LinearRegression(),
#                Lasso(),
#                ElasticNet(),
               RandomForestRegressor( n_estimators = 1000 ),
             ]

In [19]:
%%time
import time
for model in model_list :
    params = {
#                'n_estimators' : (5,1000),    # RandomForesterClassifier
             }    # grid search에서 활용할 params grid

    gs = grid_search.GridSearchCV( model,
                                   param_grid = params, n_jobs = n_jobs_cnt, cv = cv_cnt, )
    gs.fit( X, y )
    
    
    print( 'model : ', str( model ).split( '(' )[0] )
    print( 'best_score : ', gs.best_score_ )
    
    print( '=================' )
    print( 'best model : ', gs.best_estimator_ )
    print( '=================' )

model :  RandomForestRegressor
best_score :  0.766391409142
best model :  RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)
Wall time: 1min 41s


###2.4.7 test data에 대한 7개 feature 대상 encoding 수행

In [20]:
df_bend_raius_test[ 'material_id' ] = le_material_id.transform( df_bend_raius_test[ 'material_id' ] )

df_bend_raius_test[ 'end_a_1x' ] = le_end_a_1x.transform( df_bend_raius_test[ 'end_a_1x' ] )
df_bend_raius_test[ 'end_a_2x' ] = le_end_a_2x.transform( df_bend_raius_test[ 'end_a_2x' ] )

df_bend_raius_test[ 'end_x_1x' ] = le_end_x_1x.transform( df_bend_raius_test[ 'end_x_1x' ] )
df_bend_raius_test[ 'end_x_2x' ] = le_end_x_2x.transform( df_bend_raius_test[ 'end_x_2x' ] )

df_bend_raius_test[ 'end_a' ] = le_end_a.transform( df_bend_raius_test[ 'end_a' ] )
df_bend_raius_test[ 'end_x' ] = le_end_x.transform( df_bend_raius_test[ 'end_x' ] )

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydat

In [21]:
# sz_tube_assembly_id = df_bend_raius_test[ 'tube_assembly_id' ]    # id 백업

In [22]:
df_bend_raius_test.drop( removal_feature_list, axis = 1, inplace = True )    # features engineered
df_bend_raius_test.drop( [ 'tube_assembly_id', 'bend_radius' ], axis = 1, inplace = True )

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


###2.4.8 bend_radius prediction 수행

In [23]:
y_pred = gs.best_estimator_.predict( df_bend_raius_test )    # prediction 수행

In [24]:
y_pred

array([  60.40735667,   65.0112    ,   69.88295833,  179.85465   ,
        172.77525   ,  172.77525   ,  179.6409    ,   20.94137018])

###2.4.9 Prediction 결과를 tube에 value assign 수행

In [25]:
idx_list = df[ df[ 'bend_radius' ] > 9000 ].index.tolist()
idx_list

[15132, 15174, 15175, 17688, 17689, 18002, 18003, 19320]

In [26]:
# bend_radius prediction 결과를 df에 assign

jdx = 0
for idx in idx_list :
    df.loc[ idx, 'bend_radius' ] = y_pred[jdx]
    jdx += 1

###2.4.10 prediction 반영결과를 output

In [27]:
df.to_csv( './dataset/02.ml.verified.dataset/tube.tube_radius.verified.csv', index = False )
del( df )
del( df_bend_radius_train )
del( df_bend_raius_test )
del( X )
del( y )

.

##2.5 categorical feature overview
###대상 : end_a_1x, end_a_2x, end_x_1x, end_x_2x, end_a, end_x

In [28]:
df = pd.read_csv( './dataset/02.ml.verified.dataset/tube.tube_radius.verified.csv' )

In [29]:
df[ 'tube_assembly_id' ].count(),  df[ 'end_a_1x' ].count(), df[ 'end_a_1x' ].unique()    # 누락없음, 문제없음

(21198, 21198, array(['N', 'Y'], dtype=object))

In [30]:
df[ 'tube_assembly_id' ].count(),  df[ 'end_a_2x' ].count(), df[ 'end_a_2x' ].unique()    # 누락없음, 문제없음

(21198, 21198, array(['N', 'Y'], dtype=object))

In [31]:
df[ 'tube_assembly_id' ].count(),  df[ 'end_x_1x' ].count(), df[ 'end_x_1x' ].unique()     # 누락없음, 문제없음

(21198, 21198, array(['N', 'Y'], dtype=object))

In [32]:
df[ 'tube_assembly_id' ].count(),  df[ 'end_x_2x' ].count(), df[ 'end_x_2x' ].unique()     # 누락없음, 문제없음

(21198, 21198, array(['N', 'Y'], dtype=object))

In [33]:
df[ 'tube_assembly_id' ].count(),  df[ 'end_a' ].count(), df[ 'end_a' ].unique()    # 누락없음, NONE 있음

(21198,
 21198,
 array(['EF-003', 'EF-008', 'EF-021', 'NONE', 'EF-018', 'EF-005', 'EF-009',
        'EF-023', 'EF-017', 'EF-015', 'EF-012', 'EF-001', 'EF-010',
        'EF-004', 'EF-002', 'EF-013', 'EF-019', 'EF-007', 'EF-020',
        'EF-025', 'EF-016', 'EF-022', 'EF-011', 'EF-006', 'EF-014'], dtype=object))

In [34]:
df[ 'tube_assembly_id' ].count(),  df[ 'end_x' ].count(), df[ 'end_x' ].unique()    # 누락없음, NONE 있음

(21198,
 21198,
 array(['EF-003', 'EF-008', 'EF-021', 'NONE', 'EF-018', 'EF-009', 'EF-017',
        'EF-019', 'EF-015', 'EF-012', 'EF-023', 'EF-002', 'EF-013', '9999',
        'EF-007', 'EF-010', 'EF-006', 'EF-016', 'EF-022', 'EF-004',
        'EF-001', 'EF-005', 'EF-011', 'EF-025', 'EF-014', 'EF-026', 'EF-024'], dtype=object))

* 확인결과
    
    end_a_1x : 누락없음, N/Y
    
    end_a_1x : 누락없음, N/Y
    
    end_x_1x : 누락없음, N/Y
    
    end_x_2x : 누락없음, N/Y
    
    end_a : 누락없음, 25개 Category
    
    end_x : 누락없음, 9999 Unknown 존재

.

##2.5 end_x의 보완

In [35]:
df[ df[ 'end_x' ] == '9999' ]

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
958,TA-00959,SP-0028,25.40,1.65,35,1,50.8,N,N,N,N,EF-003,9999,0,0,0
8884,TA-08885,SP-0029,88.90,1.65,179,1,127.0,N,N,N,N,EF-017,9999,0,0,0
9736,TA-09737,SP-0029,88.90,1.65,68,2,101.6,N,Y,N,N,EF-009,9999,0,0,0
11386,TA-11387,SP-0029,12.70,1.65,68,6,25.4,N,Y,N,N,EF-003,9999,0,0,0
11718,TA-11719,SP-0033,127.00,1.65,34,1,127.0,Y,Y,Y,Y,EF-023,9999,0,0,0
11769,TA-11770,SP-0038,6.35,0.89,5,1,12.7,N,Y,N,N,EF-008,9999,0,0,0


In [36]:
len( df[ df[ 'end_x' ] == '9999' ] )

6

* 6개의 unknown end_x가 존재함

###2.5.1 end_x를 위한 train/ test dataset의 구성

In [37]:
# training dataset 구성시, material이  NA인 것은 제외하고 진행한다
df_end_x_train = df[ ( df[ 'end_x' ] != '9999' ) & ( df[ 'material_id' ].notnull() ) ]    # 정상 dataset을 train
df_end_x_test = df[ df[ 'end_x' ] == '9999' ]    # 9999 dataset을 test

###2.5.2 Label Encoding 작업
  * 아래의 작업은 2.4.2를 통해 필요한 feature들을 대상으로 이미 진행완료되었다고 보는 것이 합리적
  * categorical data를 모두 포함해야 하므로, df를 기반으로 label encoder를 fitting한다
  * label encoding의 대상은 materian_id, end_a_1x, end_a_2x, end_x_1x, end_x_2x, end_a, end_x 7개 feature이다
  * 단, material_id의 경우, df의 상태로는 NA가 포함되어 있으므로, 이를 제거하고 encoding 수행

###2.5.3 training data에 대한 7개 feature 대상 encoding 수행   

In [38]:
df_end_x_train[ 'material_id' ] = le_material_id.transform( df_end_x_train[ 'material_id' ] )

df_end_x_train[ 'end_a_1x' ] = le_end_a_1x.transform( df_end_x_train[ 'end_a_1x' ] )
df_end_x_train[ 'end_a_2x' ] = le_end_a_2x.transform( df_end_x_train[ 'end_a_2x' ] )

df_end_x_train[ 'end_x_1x' ] = le_end_x_1x.transform( df_end_x_train[ 'end_x_1x' ] )
df_end_x_train[ 'end_x_2x' ] = le_end_x_2x.transform( df_end_x_train[ 'end_x_2x' ] )

df_end_x_train[ 'end_a' ] = le_end_a.transform( df_end_x_train[ 'end_a' ] )
df_end_x_train[ 'end_x' ] = le_end_x.transform( df_end_x_train[ 'end_x' ] )

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydat

###2.5.4 train dataset의 encoded 상태 확인

In [39]:
df_end_x_train.head( 3 )
# 이것 활용하여, end_x == 9999를 보정하기 위한, regression model을 만들어야 함

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
0,TA-00001,9,12.70,1.65,164,5,38.10,0,0,0,0,2,3,0,0,0
1,TA-00002,1,6.35,0.71,137,8,19.05,0,0,0,0,7,8,0,0,0
2,TA-00003,1,6.35,0.71,127,7,19.05,0,0,0,0,7,8,0,0,0


In [40]:
df_end_x_train[ 'end_x' ].unique()

array([ 3,  8, 20, 26, 18,  9, 17, 19, 15, 12, 22,  2, 13,  7, 10,  6, 16,
       21,  4,  1,  5, 11, 24, 14, 25, 23], dtype=int64)

###2.5.5 Modeling 위한 X, y 생성       

In [41]:
removal_feature_list = [
#                          'material_id',
#                          'diameter',
#                          'wall',
#                          'length',
    
#                          'num_bends',
#                          'bend_radius',
    
#                          'end_a_1x',
#                          'end_a_2x',
#                          'end_x_1x',
#                          'end_x_2x',
#                          'end_a',
    
#                          'num_boss',
#                          'num_bracket',
#                          'other',
                       ]

In [42]:
df_end_x_train.drop( removal_feature_list, axis = 1, inplace = True )    # features engineered   

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [43]:
X = df_end_x_train.drop( [ 'tube_assembly_id', 'end_x' ], axis = 1 )    # X를 생성한다
y = df_end_x_train[ 'end_x' ]    # y를 생성한다   

In [44]:
X.head( 3 )

,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,num_boss,num_bracket,other
0,9,12.70,1.65,164,5,38.10,0,0,0,0,2,0,0,0
1,1,6.35,0.71,137,8,19.05,0,0,0,0,7,0,0,0
2,1,6.35,0.71,127,7,19.05,0,0,0,0,7,0,0,0


In [45]:
y.head( 10 )

0     3
1     8
2     8
3     8
4     3
5     3
6     3
7     8
8     3
9    20
Name: end_x, dtype: int64

###2.5.6 Modeling 수행

In [46]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import grid_search

In [47]:
model_list = [
#                DecisionTreeClassifier(),
               RandomForestClassifier( n_estimators = 1000 ),
             ]

In [48]:
%%time
import time
for model in model_list :
    params = {
#                'n_estimators' : (5,200),    # RandomForesterClassifier
             }    # grid search에서 활용할 params grid

    gs = grid_search.GridSearchCV( model,
                                   param_grid = params, n_jobs = n_jobs_cnt, cv = cv_cnt, )
    gs.fit( X, y )
    
    
    print( 'model : ', str( model ).split( '(' )[0] )
    print( 'best_score : ', gs.best_score_ )
    
    print( '=================' )
    print( 'best model : ', gs.best_estimator_ )
    print( '=================' )

model :  RandomForestClassifier
best_score :  0.859752307177
best model :  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Wall time: 2min 38s


C:\Anaconda3\lib\site-packages\scikit_learn-0.16.1-py3.4-win-amd64.egg\sklearn\cross_validation.py:417: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


###2.5.7 test data에 대한 7개 feature 대상 encoding 수행

In [49]:
df_end_x_test[ 'material_id' ] = le_material_id.transform( df_end_x_test[ 'material_id' ] )

df_end_x_test[ 'end_a_1x' ] = le_end_a_1x.transform( df_end_x_test[ 'end_a_1x' ] )
df_end_x_test[ 'end_a_2x' ] = le_end_a_2x.transform( df_end_x_test[ 'end_a_2x' ] )

df_end_x_test[ 'end_x_1x' ] = le_end_x_1x.transform( df_end_x_test[ 'end_x_1x' ] )
df_end_x_test[ 'end_x_2x' ] = le_end_x_2x.transform( df_end_x_test[ 'end_x_2x' ] )

df_end_x_test[ 'end_a' ] = le_end_a.transform( df_end_x_test[ 'end_a' ] )
df_end_x_test[ 'end_x' ] = le_end_x.transform( df_end_x_test[ 'end_x' ] )

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydat

In [50]:
df_end_x_test.head( 3 )

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other
958,TA-00959,2,25.4,1.65,35,1,50.8,0,0,0,0,2,0,0,0,0
8884,TA-08885,3,88.9,1.65,179,1,127.0,0,0,0,0,16,0,0,0,0
9736,TA-09737,3,88.9,1.65,68,2,101.6,0,1,0,0,8,0,0,0,0


In [51]:
df_end_x_test.drop( removal_feature_list, axis = 1, inplace = True )    # features engineered
df_end_x_test.drop( [ 'tube_assembly_id', 'end_x' ], axis = 1, inplace = True )

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


###2.5.8 bend_radius prediction 수행

In [52]:
y_pred = gs.best_estimator_.predict( df_end_x_test )    # prediction 수행

###2.5.9 Prediction 결과를 tube에 value assign 수행

In [53]:
idx_list = df[ df[ 'end_x' ] == '9999' ].index.tolist()

In [54]:
# end_x prediction 결과를 df에 assign

jdx = 0
for idx in idx_list :
    df.loc[ idx, 'end_x' ] = le_end_x.inverse_transform( y_pred[jdx] )    # 원 data 형식에 맞게
    jdx += 1

###2.5.10 prediction 반영결과를 output    

In [55]:
df.to_csv( './dataset/02.ml.verified.dataset/tube.end_x.verified.csv', index = False )
del( df )
del( df_end_x_train )
del( df_end_x_test )
del( X )
del( y )

.

##2.6 material_id의 보완

In [56]:
df = pd.read_csv( './dataset/02.ml.verified.dataset/tube.end_x.verified.csv' )

# meterial_id의 확장예측 코드
df_bill = pd.read_csv( './dataset/02.ml.verified.dataset/bill_of_materials.verified.csv' )
df_spec = pd.read_csv( './dataset/02.ml.verified.dataset/specs.verified.csv' )

In [57]:
len( df[ df[ 'material_id' ].isnull() ] )

279

In [58]:
# meterial_id의 확장예측 코드
df = df.merge( df_bill, on = 'tube_assembly_id', how = 'inner' )
df = df.merge( df_spec, on = 'tube_assembly_id', how = 'inner' )

In [59]:
# meterial_id의 확장예측 코드
for i in range( 1, 9 ) :
    df.drop( [ 'component_id_' + str( i ), 'quantity_' + str( i ) ], axis = 1, inplace = True )    

for i in range( 1, 11 ) :
    df.drop( [ 'spec' + str( i ) ], axis = 1, inplace = True )    

* 279개의 material_id가 누락됨

###2.6.1 material_id를 위한 train/ test dataset의 구성

In [60]:
df_material_id_train = df[ df[ 'material_id' ].notnull() ]    # 정상 dataset을 train
df_material_id_test = df[ df[ 'material_id' ].isnull() ]    # null dataset을 test

###2.6.2 Label Encoding 작업
* 아래의 작업은 2.4.2를 통해 필요한 feature들을 대상으로 이미 진행완료되었다고 보는 것이 합리적
* categorical data를 모두 포함해야 하므로, df를 기반으로 label encoder를 fitting한다
* label encoding의 대상은 materian_id, end_a_1x, end_a_2x, end_x_1x, end_x_2x, end_a, end_x 7개 feature이다
* 단, material_id의 경우, df의 상태로는 NA가 포함되어 있으므로, 이를 제거하고 encoding 수행

###2.6.3 training data에 대한 7개 feature 대상 encoding 수행

In [61]:
df_material_id_train[ 'material_id' ] = le_material_id.transform( df_material_id_train[ 'material_id' ] )

df_material_id_train[ 'end_a_1x' ] = le_end_a_1x.transform( df_material_id_train[ 'end_a_1x' ] )
df_material_id_train[ 'end_a_2x' ] = le_end_a_2x.transform( df_material_id_train[ 'end_a_2x' ] )

df_material_id_train[ 'end_x_1x' ] = le_end_x_1x.transform( df_material_id_train[ 'end_x_1x' ] )
df_material_id_train[ 'end_x_2x' ] = le_end_x_2x.transform( df_material_id_train[ 'end_x_2x' ] )

df_material_id_train[ 'end_a' ] = le_end_a.transform( df_material_id_train[ 'end_a' ] )
df_material_id_train[ 'end_x' ] = le_end_x.transform( df_material_id_train[ 'end_x' ] )

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydat

In [62]:
# meterial_id의 확장예측 코드
dic = { 'Y' : 1, 'N' : 0 }
df.replace( dic, inplace = True )

###2.6.4 train dataset의 encoded 상태 확인

In [63]:
df_material_id_train.head( 3 )
# 이것 활용하여, end_x == 9999를 보정하기 위한, regression model을 만들어야 함

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,...,SP-0087,SP-0088,SP-0089,SP-0090,SP-0091,SP-0092,SP-0093,SP-0094,SP-0095,SP-0096
0,TA-00001,9,12.70,1.65,164,5,38.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TA-00002,1,6.35,0.71,137,8,19.05,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TA-00003,1,6.35,0.71,127,7,19.05,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
df_material_id_train[ 'material_id' ].unique()

array([ 9,  1,  3, 13, 17,  2, 14, 11,  0,  7, 12, 16,  4,  5, 18, 15,  8,
        6, 10], dtype=int64)

###2.6.5 Modeling 위한 X, y 생성

In [65]:
removal_feature_list = [
#                          'diameter',
#                          'wall',
#                          'length',
    
#                          'num_bends',
#                          'bend_radius',
    
#                          'end_a_1x',
#                          'end_a_2x',
#                          'end_x_1x',
#                          'end_x_2x',
#                          'end_a',
#                          'end_x',
    
#                          'num_boss',
#                          'num_bracket',
#                          'other',
                       ]

In [66]:
df_material_id_train.drop( removal_feature_list, axis = 1, inplace = True )    # features engineered   

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [67]:
X = df_material_id_train.drop( [ 'tube_assembly_id', 'material_id' ], axis = 1 )    # X를 생성한다
y = df_material_id_train[ 'material_id' ]    # y를 생성한다   

In [68]:
X.head( 3 )

,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,...,SP-0087,SP-0088,SP-0089,SP-0090,SP-0091,SP-0092,SP-0093,SP-0094,SP-0095,SP-0096
0,12.70,1.65,164,5,38.10,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
1,6.35,0.71,137,8,19.05,0,0,0,0,7,...,0,0,0,0,0,0,0,0,0,0
2,6.35,0.71,127,7,19.05,0,0,0,0,7,...,0,0,0,0,0,0,0,0,0,0


In [69]:
y.head( 10 )

0     9
1     1
2     1
3     1
4     3
5     3
6     9
7    13
8     3
9    17
Name: material_id, dtype: int64

###2.6.6 Modeling 수행

In [70]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import grid_search

In [71]:
model_list = [
#                DecisionTreeClassifier(),
               RandomForestClassifier( n_estimators = 1000 ),
             ]

In [72]:
%%time
import time
for model in model_list :
    params = {
#                'n_estimators' : (5,200),    # RandomForesterClassifier
             }    # grid search에서 활용할 params grid

    gs = grid_search.GridSearchCV( model,
                                   param_grid = params, n_jobs = n_jobs_cnt, cv = cv_cnt, )
    gs.fit( X, y )
    
    
    print( 'model : ', str( model ).split( '(' )[0] )
    print( 'best_score : ', gs.best_score_ )
    
    print( '=================' )
    print( 'best model : ', gs.best_estimator_ )
    print( '=================' )

model :  RandomForestClassifier
best_score :  0.828337874659
best model :  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Wall time: 3min 21s


C:\Anaconda3\lib\site-packages\scikit_learn-0.16.1-py3.4-win-amd64.egg\sklearn\cross_validation.py:417: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


###2.6.7 test data에 대한 7개 feature 대상 encoding 수행

In [73]:
df_material_id_test[ 'end_a_1x' ] = le_end_a_1x.transform( df_material_id_test[ 'end_a_1x' ] )
df_material_id_test[ 'end_a_2x' ] = le_end_a_2x.transform( df_material_id_test[ 'end_a_2x' ] )

df_material_id_test[ 'end_x_1x' ] = le_end_x_1x.transform( df_material_id_test[ 'end_x_1x' ] )
df_material_id_test[ 'end_x_2x' ] = le_end_x_2x.transform( df_material_id_test[ 'end_x_2x' ] )

df_material_id_test[ 'end_a' ] = le_end_a.transform( df_material_id_test[ 'end_a' ] )
df_material_id_test[ 'end_x' ] = le_end_x.transform( df_material_id_test[ 'end_x' ] )

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentatio

In [74]:
df_material_id_test.head( 3 )

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,...,SP-0087,SP-0088,SP-0089,SP-0090,SP-0091,SP-0092,SP-0093,SP-0094,SP-0095,SP-0096
1701,TA-01702,NaN,17.48,2.41,115,3,63.5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2003,TA-02004,NaN,17.30,2.30,30,0,0.0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3359,TA-03360,NaN,19.05,2.11,151,4,50.8,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
df_material_id_test.drop( removal_feature_list, axis = 1, inplace = True )    # features engineered
df_material_id_test.drop( [ 'tube_assembly_id', 'material_id' ], axis = 1, inplace = True )

C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Anaconda3\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app


###2.6.8 material_id prediction 수행

In [76]:
y_pred = gs.best_estimator_.predict( df_material_id_test )    # prediction 수행

###2.6.9 Prediction 결과를 tube에 value assign 수행

In [77]:
idx_list = df[ df[ 'material_id' ].isnull() ].index.tolist()

In [78]:
# material_id prediction 결과를 df에 assign

jdx = 0
for idx in idx_list :
    df.loc[ idx, 'material_id' ] = le_material_id.inverse_transform( y_pred[jdx] )    # 원 data 형식에 맞게
    jdx += 1

In [81]:
df.columns.values

array(['tube_assembly_id', 'material_id', 'diameter', 'wall', 'length',
       'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x',
       'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other',
       'weight_sum', 'length_1', 'length_2', 'length_3', 'length_4',
       'length_5', 'length_6', 'length_7', 'length_8', 'length_sum',
       'uniqueness_count', 'orientation_count', 'adaptor_sum', 'boss_sum',
       'elbow_sum', 'float_sum', 'hfl_sum', 'nut_sum', 'other_sum',
       'sleeve_sum', 'straight_sum', 'tee_sum', 'threaded_sum',
       'adaptor_weight_sum', 'boss_weight_sum', 'elbow_weight_sum',
       'float_weight_sum', 'hfl_weight_sum', 'nut_weight_sum',
       'other_weight_sum', 'sleeve_weight_sum', 'straight_weight_sum',
       'tee_weight_sum', 'threaded_weight_sum', 'spec_type_count',
       'SP-0001', 'SP-0002', 'SP-0003', 'SP-0004', 'SP-0005', 'SP-0006',
       'SP-0007', 'SP-0008', 'SP-0009', 'SP-0010', 'SP-0011', 'SP-0012',
       'SP-0013', 'SP-00

In [83]:
# meterial_id의 확장예측 코드
df.drop( [   'weight_sum', 'length_1', 'length_2', 'length_3', 'length_4',
       'length_5', 'length_6', 'length_7', 'length_8', 'length_sum',
             'uniqueness_count', 'orientation_count',
             'adaptor_sum', 'boss_sum', 'elbow_sum', 'float_sum', 'hfl_sum',
             'nut_sum', 'other_sum', 'sleeve_sum', 'straight_sum', 'tee_sum',
             'threaded_sum', 'adaptor_weight_sum', 'boss_weight_sum',
             'elbow_weight_sum', 'float_weight_sum', 'hfl_weight_sum',
             'nut_weight_sum', 'other_weight_sum', 'sleeve_weight_sum',
             'straight_weight_sum', 'tee_weight_sum', 'threaded_weight_sum',
             'spec_type_count', 'SP-0001', 'SP-0002', 'SP-0003', 'SP-0004',
             'SP-0005', 'SP-0006', 'SP-0007', 'SP-0008', 'SP-0009', 'SP-0010',
             'SP-0011', 'SP-0012', 'SP-0013', 'SP-0014', 'SP-0015', 'SP-0016',
             'SP-0017', 'SP-0018', 'SP-0019', 'SP-0020', 'SP-0021', 'SP-0022',
             'SP-0023', 'SP-0024', 'SP-0025', 'SP-0026', 'SP-0027', 'SP-0028',
             'SP-0029', 'SP-0030', 'SP-0031', 'SP-0032', 'SP-0033', 'SP-0034',
             'SP-0035', 'SP-0036', 'SP-0037', 'SP-0038', 'SP-0039', 'SP-0040',
             'SP-0041', 'SP-0042', 'SP-0043', 'SP-0044', 'SP-0045', 'SP-0046',
             'SP-0047', 'SP-0048', 'SP-0049', 'SP-0050', 'SP-0051', 'SP-0052',
             'SP-0053', 'SP-0054', 'SP-0055', 'SP-0056', 'SP-0057', 'SP-0058',
             'SP-0059', 'SP-0060', 'SP-0061', 'SP-0062', 'SP-0063', 'SP-0064',
             'SP-0065', 'SP-0066', 'SP-0067', 'SP-0068', 'SP-0069', 'SP-0070',
             'SP-0071', 'SP-0072', 'SP-0073', 'SP-0074', 'SP-0075', 'SP-0076',
             'SP-0077', 'SP-0078', 'SP-0079', 'SP-0080', 'SP-0081', 'SP-0082',
             'SP-0083', 'SP-0084', 'SP-0085', 'SP-0086', 'SP-0087', 'SP-0088',
             'SP-0089', 'SP-0090', 'SP-0091', 'SP-0092', 'SP-0093', 'SP-0094',
             'SP-0095', 'SP-0096' ], axis = 1, inplace = True )

ValueError: labels ['weight_sum' 'length_1' 'length_2' 'length_3' 'length_4' 'length_5'
 'length_6' 'length_7' 'length_8' 'length_sum' 'uniqueness_count'
 'orientation_count' 'adaptor_sum' 'boss_sum' 'elbow_sum' 'float_sum'
 'hfl_sum' 'nut_sum' 'other_sum' 'sleeve_sum' 'straight_sum' 'tee_sum'
 'threaded_sum' 'adaptor_weight_sum' 'boss_weight_sum' 'elbow_weight_sum'
 'float_weight_sum' 'hfl_weight_sum' 'nut_weight_sum' 'other_weight_sum'
 'sleeve_weight_sum' 'straight_weight_sum' 'tee_weight_sum'
 'threaded_weight_sum' 'spec_type_count' 'SP-0001' 'SP-0002' 'SP-0003'
 'SP-0004' 'SP-0005' 'SP-0006' 'SP-0007' 'SP-0008' 'SP-0009' 'SP-0010'
 'SP-0011' 'SP-0012' 'SP-0013' 'SP-0014' 'SP-0015' 'SP-0016' 'SP-0017'
 'SP-0018' 'SP-0019' 'SP-0020' 'SP-0021' 'SP-0022' 'SP-0023' 'SP-0024'
 'SP-0025' 'SP-0026' 'SP-0027' 'SP-0028' 'SP-0029' 'SP-0030' 'SP-0031'
 'SP-0032' 'SP-0033' 'SP-0034' 'SP-0035' 'SP-0036' 'SP-0037' 'SP-0038'
 'SP-0039' 'SP-0040' 'SP-0041' 'SP-0042' 'SP-0043' 'SP-0044' 'SP-0045'
 

In [84]:
df.columns.values

array(['tube_assembly_id', 'material_id', 'diameter', 'wall', 'length',
       'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x',
       'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other'], dtype=object)

###2.6.10 prediction 반영결과를 output

In [85]:
df.to_csv( './dataset/02.ml.verified.dataset/tube.material_id.verified.csv', index = False )
del( df )
del( df_material_id_train )
del( df_material_id_test )
del( X )
del( y )

.

##2.7 tube feature 확인

In [86]:
df = pd.read_csv( './dataset/02.ml.verified.dataset/tube.material_id.verified.csv' )

In [87]:
# bend nub으로 인한 tube의 복잡도 추정 feature의 추가
df[ 'bend_num_by_radius' ] = df[ 'num_bends' ] * df[ 'bend_radius' ]

In [88]:
df.to_csv( './dataset/02.ml.verified.dataset/tube.material_id.verified.csv', index = False )

In [89]:
df.count()

tube_assembly_id      21198
material_id           21198
diameter              21198
wall                  21198
length                21198
num_bends             21198
bend_radius           21198
end_a_1x              21198
end_a_2x              21198
end_x_1x              21198
end_x_2x              21198
end_a                 21198
end_x                 21198
num_boss              21198
num_bracket           21198
other                 21198
bend_num_by_radius    21198
dtype: int64

In [90]:
df.describe()

,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,num_boss,num_bracket,other,bend_num_by_radius
count,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000,21198.000000
mean,23.687764,1.578402,100.337469,3.748137,45.620664,0.050948,0.150156,0.038730,0.127606,0.044485,0.006746,0.022644,156.837303
std,25.396277,0.745749,70.520780,2.265405,35.028261,0.219897,0.357232,0.192955,0.333659,0.260737,0.100015,0.199676,120.463532
min,3.180000,0.710000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.520000,0.890000,48.000000,2.000000,19.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,76.200000
50%,15.880000,1.650000,86.000000,3.000000,38.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,127.000000
75%,25.400000,1.650000,135.000000,5.000000,50.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,203.200000
max,203.200000,7.900000,1333.000000,17.000000,565.000000,1.000000,1.000000,1.000000,1.000000,5.000000,5.000000,8.000000,2706.000000


In [91]:
df[ df[ 'bend_radius' ] > 9000 ]    # bend_radius를 prediction해야하는 test dataset이다

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other,bend_num_by_radius


In [92]:
df[ df[ 'end_x' ] == '9999' ]    # bend_radius를 prediction해야하는 test dataset이다

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other,bend_num_by_radius


In [93]:
df[ df[ 'material_id' ].isnull() ]

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other,bend_num_by_radius


##완료

In [94]:
del( df )
# del( df_bill )
# del( df_spec )